In [1]:
import findspark
findspark.init()
# Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
# Create SparkSession
spark = SparkSession.builder \
            .appName('SparkByExamples.com') \
            .getOrCreate()
spark

In [15]:
sourceData=[(1,"A"),(2,"B"),(3,"C"),(4,"D")]
targetData=[(1,"A"),(2,"B"),(4,"X"),(5,"F")]
columns=['id','name']

source=spark.createDataFrame(sourceData,schema=columns)
target=spark.createDataFrame(targetData,schema=columns)

source.show()
target.show()

+---+----+
| id|name|
+---+----+
|  1|   A|
|  2|   B|
|  3|   C|
|  4|   D|
+---+----+

+---+----+
| id|name|
+---+----+
|  1|   A|
|  2|   B|
|  4|   X|
|  5|   F|
+---+----+



In [10]:
source.join(target,(source.id==target.id) & (source.name==target.name),"left_anti").show()

+---+----+
| id|name|
+---+----+
|  3|   C|
|  4|   D|
+---+----+



In [21]:
joindf=source.join(target,(source.id==target.id),"full").\
       select(source.id.alias("sourceid"),source.name.alias("sourcename"),\
       target.id.alias("targetid"),target.name.alias("targetname"))
joindf.show()

+--------+----------+--------+----------+
|sourceid|sourcename|targetid|targetname|
+--------+----------+--------+----------+
|       1|         A|       1|         A|
|       2|         B|       2|         B|
|       3|         C|    null|      null|
|       4|         D|       4|         X|
|    null|      null|       5|         F|
+--------+----------+--------+----------+



In [34]:
joindf.select(coalesce(col("sourceid"),col("targetid")).alias("id"),\
              when((col("sourceid")==col("targetid")) & (col("sourcename")!=col("targetname")), "Mismatched").
              when(col("sourceid").isNull(),"New in Source").\
              when(col("targetid").isNull(),"New in Target").\
              otherwise("Matched").alias("Comments")).show()

+---+-------------+
| id|     Comments|
+---+-------------+
|  1|      Matched|
|  2|      Matched|
|  3|New in Target|
|  4|   Mismatched|
|  5|New in Source|
+---+-------------+



In [45]:
#Missing Numbers
ip=[(1,),(3,),(5,),(7,),(9,),(10,)]
df=spark.createDataFrame(ip,['id'])
full_range=spark.range(1,11).toDF("id")

In [46]:
full_range.join(df,"id","left").where(df.id.isNull()).show()

+---+
| id|
+---+
|  2|
|  4|
|  6|
|  8|
+---+



In [47]:
#Products whose total sales revenue has increased every year
sales_data=[(1,2019,1000),(1,2020,1200),(1,2021,1100),(2,2019,500),(2,2020,600),(2,2021,900),(3,2019,300),(3,2020,450),(3,2021,400)]
sales_col=['product_id','year','total_sales_revenue']

product_data=[(1,"Laptop","Electronics"),(2,"Jeans","Clothing"),(3,"Chair","Home_Appliance")]
product_col=['product_id','product_name','category']

sales=spark.createDataFrame(sales_data,schema=sales_col)
product=spark.createDataFrame(product_data,schema=product_col)

sales.show()
product.show()

+----------+----+-------------------+
|product_id|year|total_sales_revenue|
+----------+----+-------------------+
|         1|2019|               1000|
|         1|2020|               1200|
|         1|2021|               1100|
|         2|2019|                500|
|         2|2020|                600|
|         2|2021|                900|
|         3|2019|                300|
|         3|2020|                450|
|         3|2021|                400|
+----------+----+-------------------+

+----------+------------+--------------+
|product_id|product_name|      category|
+----------+------------+--------------+
|         1|      Laptop|   Electronics|
|         2|       Jeans|      Clothing|
|         3|       Chair|Home_Appliance|
+----------+------------+--------------+



In [62]:
from pyspark.sql.window import Window
windowspec=Window.partitionBy("product_id").orderBy("year")
df=sales.withColumn("previous_revenue",lag("total_sales_revenue").over(windowspec)).\
      withColumn("revenue_diff", col("total_sales_revenue")-col("previous_revenue"))
df.groupBy("product_id").agg(min("revenue_diff").alias("min_diff")).filter(col("min_diff")>0).show()

+----------+--------+
|product_id|min_diff|
+----------+--------+
|         2|     100|
+----------+--------+



In [26]:
#Flatten JSON Data
df=spark.read.option("multiLine", "true").option("mode", "PERMISSIVE").json("coaching.json")
df.withColumn("branches",explode("branches")).\
   select("Institute_Name","Course_type","Head_Office_Contact","branches.*").\
   show(truncate=False)

+-------------------+-----------+-------------------+------+-----------+-------+
|Institute_Name     |Course_type|Head_Office_Contact|City  |State      |address|
+-------------------+-----------+-------------------+------+-----------+-------+
|ABC_Coaching_Center|Best_seller|8787878787         |Mumbai|Maharashtra|XYZ    |
|ABC_Coaching_Center|Best_seller|8787878787         |Surat |Gujrat     |PQRX   |
+-------------------+-----------+-------------------+------+-----------+-------+



In [39]:
#Total number of hours each employee was inside office
import time
import datetime

_data = [
(11114,'08:30:00',"I"),
(11114,'10:30:00','O'),
(11114,'11:30:00','I'),
(11114,'15:30:00','O'),
(11115,'09:30:00','I'),
(11115,'17:30:00','O')
]
from pyspark.sql.types import StructType,StructField, TimestampType, LongType, StringType
_schema = StructType([
  StructField('emp_id', LongType(), True),
  StructField('punch_time', StringType(), True),
  StructField('flag', StringType(), True)
])
df = spark.createDataFrame(data = _data,schema=_schema)
df.withColumn("punch_time",to_timestamp(col("punch_time"))).show()

+------+-------------------+----+
|emp_id|         punch_time|flag|
+------+-------------------+----+
| 11114|2024-03-26 08:30:00|   I|
| 11114|2024-03-26 10:30:00|   O|
| 11114|2024-03-26 11:30:00|   I|
| 11114|2024-03-26 15:30:00|   O|
| 11115|2024-03-26 09:30:00|   I|
| 11115|2024-03-26 17:30:00|   O|
+------+-------------------+----+



In [ ]:
from pyspark.sql.window import Window
windowspec=Window.partitionBy("emp_id")
window